In [257]:
import numpy as np
import pandas as pd
import itertools
from pulp import *
from tqdm.notebook import tqdm

In [258]:
def pareto_dominance(x, y, **kwargs):
    return np.greater_equal(x,y).all() and np.greater(x,y).any()

In [259]:
class Preference_DB:
    
    def __init__(self, items):
        self.items = items
        self.subsets = {}
    
    def register(self, subset, performance):
        pass
    
    def getRelation(self, relation_f, **kwargs):
        R = []
        print("Getting dominance")
        pbar = tqdm(total=len(self.subsets)*len(self.subsets)) 
        for k in self.subsets:
            for j in self.subsets:
                if(relation_f(self.subsets[k], self.subsets[j], **kwargs)):
                    if((k,j) not in R):
                        R.append((k,j))
                pbar.update(1)
        return R
    
    def compute_subsets_from_u(self, n_subsets, u):
        """
        u: S -> u(S)
        where S is a subset and S1 dominates S2 if and only if u(S1) >= u(S2)
        """
        print("Sampling subsets")
        for i in tqdm(range(n_subsets)):
            k = np.random.randint(1, self.items.shape[0])
            subset = tuple(sorted(set(np.random.choice(self.items, k))))
            if not (subset in self.subsets):
                self.subsets[subset] = np.array([u(subset)])


In [260]:
class Additive_Utility_Function:
    
    def __init__(self, items_set):
        self.items_set = items_set
        self.theta = {}
    
    def add_coefficient(self, subset, coefficient):
        self.theta[tuple(sorted(subset))] = coefficient
        
    def sample_random_coeffs(self, k ):
        print("Sampling random coefs")
        for i in tqdm(range(1,k+1)):
            coeffs = itertools.combinations(self.items_set, i)
            for c in coeffs:
                r = np.random.random()
                if c not in self.theta:
                    self.theta[c] = r
    
    def __call__(self, subset):
        s = 0
        for k in self.theta:
            if all(j in subset for j in k):
                s += self.theta[k]
        return s

In [261]:
U = Additive_Utility_Function([1, 2, 3, 4])
U.sample_random_coeffs(2)
U([1,3])

Sampling random coefs


  0%|          | 0/2 [00:00<?, ?it/s]

1.74924575484784

In [262]:
U.theta

{(1,): 0.7916968904089882,
 (2,): 0.0019826956849219313,
 (3,): 0.4697410810291759,
 (4,): 0.9902759287415382,
 (1, 2): 0.5255630095912255,
 (1, 3): 0.487807783409676,
 (1, 4): 0.6688381070636118,
 (2, 3): 0.3836636266133291,
 (2, 4): 0.06928916125852791,
 (3, 4): 0.7618784716329937}

In [355]:
class Utility_Fitter:
    
    def __init__(self):
        pass
    
    def fit(self, preferences):
        pass
    
    def predict(self, subset):
        pass

class Additive_Utility_Fitter:
    
    def __init__(self, items_set, k):
        super().__init__()
        self.k = k
        self.problem = LpProblem("Utility_Fitting")
        self.theta = {}
        self.gap = {}
        self.items_set = items_set
        self.preferences = []
        self.max_gap = 1

    
    def get_utility_exp(self, subset):
        exp = 0
        for c in self.theta:
            if(all(i in subset for i in c)):
                exp += self.theta[c]
        return exp
    
    def evaluate(self, subset):
        s = 0
        for k in self.theta:
            if all(j in subset for j in subset):
                s += value(self.theta[k])
        return s
    
    def compare(self, preferences, subset1 ,subset2):
        self.preferences = preferences
        print("Creating variables")
        for i in tqdm(range(1,self.k+1)):
            coeffs = itertools.combinations(self.items_set, i)
            for c in coeffs:
                r = np.random.normal(0, 1)
                if c not in self.theta:
                    self.theta[c] = LpVariable("v_"+"_".join([str(a) for a in c])) 
                    self.theta[c].setInitialValue(0)
                    
        print("Creating constraints:")
        for x,y in tqdm(self.preferences):
            self.gap[(x,y)] = LpVariable(name = "p_"+"".join(map(str,x)) + "_" + "".join(map(str,y)))
            self.problem += (self.gap[(x,y)] >= self.max_gap)
            cst = (self.get_utility_exp(x) >= self.get_utility_exp(y) + self.gap[(x,y)])
            print("cst: ", cst)
            print("p: ",(self.gap[(x,y)] >= self.max_gap))


        obj = -(self.get_utility_exp(subset1) - self.get_utility_exp(subset2))
        
        print("obj: ", obj)
        self.problem +=obj
        t = time()
        self.problem.solve()
        t = time() - t
        print(f"Solved in : {t} s")
        print("Status:", LpStatus[self.problem.status])
        
    
    def fit(self, preferences):
        self.preferences = preferences
        print("Creating variables")
        for i in tqdm(range(1,self.k+1)):
            coeffs = itertools.combinations(self.items_set, i)
            for c in coeffs:
                r = np.random.normal(0, 1)
                if c not in self.theta:
                    self.theta[c] = LpVariable("v_"+"_".join([str(a) for a in c])) 
                    self.theta[c].setInitialValue(0)
        print("Creating constraints:")
        for x,y in tqdm(self.preferences):
            self.gap[(x,y)] = LpVariable(name = "p_"+"".join(map(str,x)) + "_" + "".join(map(str,y)))
            self.problem += (self.gap[(x,y)] >= self.max_gap)
            cst = (self.get_utility_exp(x) >= self.get_utility_exp(y) + self.gap[(x,y)])
            print("cst: ", cst)
            print("p: ",(self.gap[(x,y)] >= self.max_gap))
            self.problem += cst
        
        self.problem += sum(self.gap.values())
        t = time()
        self.problem.solve()
        t = time() - t
        print(f"Solved in : {t} s")
        print("Status:", LpStatus[self.problem.status])
        for k in self.theta:
            if(value(self.theta[k]) != 0):
                print(self.theta[k], ": ", value(self.theta[k]))
                
    def print_gaps(self):
        for c in self.gap:
            print(self.gap[c], " : ", value(self.gap[c]))
        pass

In [356]:
DB = Preference_DB(np.arange(3))
U = Additive_Utility_Function(DB.items)
U.sample_random_coeffs(2)
DB.compute_subsets_from_u(10,U)
pref_list = DB.getRelation(pareto_dominance)
pref_list

Sampling random coefs


  0%|          | 0/2 [00:00<?, ?it/s]

Sampling subsets


  0%|          | 0/10 [00:00<?, ?it/s]

Getting dominance


  0%|          | 0/16 [00:00<?, ?it/s]

[((1,), (2,)),
 ((0,), (2,)),
 ((0,), (1,)),
 ((0, 2), (2,)),
 ((0, 2), (1,)),
 ((0, 2), (0,))]

In [359]:
AD = Additive_Utility_Fitter(DB.items, k=2)
AD.compare([((1,), (0, ))], (1,) , (0,))
AD.print_gaps()

Creating variables


  0%|          | 0/2 [00:00<?, ?it/s]

Creating constraints:


  0%|          | 0/1 [00:00<?, ?it/s]

cst:  -p_1_0 - v_0 + v_1 >= 0
p:  p_1_0 >= 1
obj:  v_0 - v_1
Solved in : 0.018254518508911133 s
Status: Unbounded
p_1_0  :  1.0


In [360]:
AD = Additive_Utility_Fitter(DB.items, k=2)
AD.fit(pref_list)

Creating variables


  0%|          | 0/2 [00:00<?, ?it/s]

Creating constraints:


  0%|          | 0/6 [00:00<?, ?it/s]

cst:  -p_1_2 + v_1 - v_2 >= 0
p:  p_1_2 >= 1
cst:  -p_0_2 + v_0 - v_2 >= 0
p:  p_0_2 >= 1
cst:  -p_0_1 + v_0 - v_1 >= 0
p:  p_0_1 >= 1
cst:  -p_02_2 + v_0 + v_0_2 + 0*v_2 >= 0
p:  p_02_2 >= 1
cst:  -p_02_1 + v_0 + v_0_2 - v_1 + v_2 >= 0
p:  p_02_1 >= 1
cst:  -p_02_0 + 0*v_0 + v_0_2 + v_2 >= 0
p:  p_02_0 >= 1
Solved in : 0.017202377319335938 s
Status: Optimal
v_0 :  2.0
v_1 :  1.0
v_0_2 :  1.0


In [361]:
for x,y in pref_list:
    x_val = value(AD.get_utility_exp(x))
    y_val = value(AD.get_utility_exp(y))
    if(not x_val > y_val):
        print(x, " : ",x_val, " > ", y , " : ", y_val)